In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale
from scipy import signal

from matplotlib.pyplot import plot
import matplotlib.pyplot as plt
%matplotlib inline
%run 'tf.py'

In [6]:
%run 'tf.py'

def shift_y(x, y, spacing_fac=5):
    return x, y + x * spacing_fac

def move_circle(c, xmax, div_fac, turns=5, spacing_fac=0., x_offset=0., turn_dir=-1, angle_offset=0):
    cx = c.getAttribute('cx')
    cy = c.getAttribute('cy')

    nx, ny = shift_y(float(cx) + x_offset, float(cy), spacing_fac)
    x, y = tf_spiral_single(nx, ny, turns, xmax, turn_dir, angle_offset)

    c.setAttribute('cx', x / div_fac)
    c.setAttribute('cy', y / div_fac)
    return c

def move_line(l, xmax, div_fac, turns=5, spacing_fac=0, x_offset=0, turn_dir=-1, angle_offset=0):
    x1 = l.getAttribute('x1')
    y1 = l.getAttribute('y1')
    x2 = l.getAttribute('x2')
    y2 = l.getAttribute('y2')

    nx1, ny1 = shift_y(float(x1) + x_offset, float(y1), spacing_fac)
    nx2, ny2 = shift_y(float(x2) + x_offset, float(y2), spacing_fac)
    
    x1_t, y1_t = tf_spiral_single(nx1, ny1, turns, xmax, turn_dir, angle_offset)
    x2_t, y2_t = tf_spiral_single(nx2, ny2, turns, xmax, turn_dir, angle_offset)

    l.setAttribute('x1', x1_t / div_fac)
    l.setAttribute('y1', y1_t / div_fac)
    l.setAttribute('x2', x2_t / div_fac)
    l.setAttribute('y2', y2_t / div_fac)
    return l

def yshift_circle(c, fac):
    cy = float(c.getAttribute('cy'))
    c.setAttribute('cy', str(cy * fac))
    return c

def yshift_line(c, fac):
    y1 = float(l.getAttribute('y1'))
    y2 = float(l.getAttribute('y2'))
    l.setAttribute('y1', str(y1 * fac))
    l.setAttribute('y2', str(y2 * fac))
    return c

def replace_move_line(l, rad, xmax, div_fac, turns=5, spacing_fac=0, x_offset=0, turn_dir=-1, angle_offset=0):
    pl = pysvg.shape.Polyline()
    pl.setAttribute('stroke-width', l.getAttribute('stroke-width'))
    pl.setAttribute('stroke', l.getAttribute('stroke'))
    
    # get start end end point of segment
    p1 = (float(l.getAttribute('x1')), float(l.getAttribute('y1')))
    p2 = (float(l.getAttribute('x2')), float(l.getAttribute('y2')))
    # get direction vector of segment
    v = (p2[0] - p1[0], p2[1] - p1[1])
    # get length of segment 
    length = math.sqrt(v[0] ** 2 + v[1] ** 2)
    
    # if line is shorter than radius
    if length <= rad:
        pl.set_points(f'{p1[0]},{p1[1]} {p2[0]},{p2[1]}')
        return pl

    # get normalized vector of segment
    v1 = (v[0] / length, v[1] / length)

    points = [p1]
    new_len = 0
    prev_p = p1
    while new_len < length:
        # check if rest of line is long enough to add rad
        if length >= new_len + rad:
            add = rad
        else:
            add = length - new_len

        # create new point with needed distance to previous one
        new_p = (prev_p[0] + v1[0] * add, prev_p[1] + v1[1] * add)
        new_len += add
        prev_p = new_p
        points.append(new_p)

    # move points (same operations as move_circle and move_line use)
    nps = [shift_y(float(p[0]) + x_offset, float(p[1]), spacing_fac) for p in points]
    npm = [tf_spiral_single(p[0], p[1], turns, xmax, turn_dir, angle_offset) for p in nps]
    npd = [(p[0] / div_fac, p[1] / div_fac) for p in npm]

    def join_points(points):
        res = ""
        for p in points:
            res += f'{p[0]},{p[1]} '
        return res.strip(' ')

    pl.set_points(join_points(npd))
    return pl

def flatten(li):
    return [item for sublist in li for item in sublist]

In [35]:
#f = pysvg.parser.parse('./svg/chart_reduced_new-ordering2-manual.svg')
f = pysvg.parser.parse('./svg/chart_reduced-reverse2.svg')
circles = f.getElementsByType(pysvg.shape.Circle)
lines = f.getElementsByType(pysvg.shape.Line)

# load clear svg
f3 = pysvg.parser.parse('./svg/blank.svg')

rect = pysvg.Rect(x=-500, y=-500, width='100%', height='100%')
rect.setAttribute('style', 'fill:#363636')
f3.addElement(rect)

# get maxima
xmax1 = max([float(c.getAttribute('cx')) for c in circles])
xmax2 = max([float(l.getAttribute('x1')) for l in lines])
xmax3 = max([float(l.getAttribute('x2')) for l in lines])
xmax = max([xmax1, xmax2, xmax3])

ymax1 = max([float(c.getAttribute('cy')) for c in circles])
ymax2 = max([float(l.getAttribute('y1')) for l in lines])
ymax3 = max([float(l.getAttribute('y2')) for l in lines])
ymax = max([ymax1, ymax2, ymax3])

# transform params
# 5 turns: 16
fac = 3.
div_fac = 440
turns = 1.2
spacing_fac = 0.35
x_offset = xmax * 0.5
turn_dir = -1  # cw: 1, ccw: -1
angle_offset = np.pi * -0.8  # in radians
rad = 3.0 # for polyline replacement

# create circle group
circle_group = pysvg.structure.G()
circle_group.setAttribute('stroke', '#fffdfa')
circle_group.setAttribute('stroke-width', '0.5')

for c in circles:
    # spread circles in y-direction
    shifted = yshift_circle(c, fac)
    # tf to spiral
    circle = move_circle(c, xmax, div_fac, turns, spacing_fac, x_offset, turn_dir, angle_offset)
    circle_group.addElement(circle)

# create line group
#line_group = pysvg.structure.G()
#line_group.setAttribute('stroke-opacity', '1')
#line_group.setAttribute('stroke-linecap', 'round')
shifted_lines = []
for l in lines:
    # spread line-points in y-direction
    shifted_lines.append(yshift_line(l, fac))
    # tf to spiral
    #line = move_line(l, xmax, div_fac, turns, spacing_fac, x_offset, turn_dir, angle_offset)
    #line_group.addElement(line)

# create polyline group
pline_group = pysvg.structure.G()
pline_group.setAttribute('fill', 'none')
plines = [replace_move_line(l, rad, xmax, div_fac, turns, spacing_fac, x_offset, turn_dir, angle_offset) for l in shifted_lines]

for pl in plines:
    pline_group.addElement(pl)

#f3.addElement(line_group)
f3.addElement(pline_group)
f3.addElement(circle_group)

# set viewBox
#f3.setAttribute('viewBox', '-500 -500 1000 1000')

# save file
f3.save('./svg/graph.svg')

### Todo:
- filter one-time characters
- spiral should start on the outside
- logarithmic scale on x-coords